In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import inout


In [ ]:
# Determine rank of micro profiles data
micro_profiles = inout.get_micro_profiles_data(1).T
micro_profiles = np.delete(micro_profiles, 51, 0)
micro_profiles = np.delete(micro_profiles, 0, 0)

np.linalg.matrix_rank(np.kron(micro_profiles, micro_profiles))


In [ ]:
def get_data(subject_id):
    sc = inout.get_schaefer100_sc('', subject_id)[14:, 14:]
    micro_profiles = inout.get_micro_profiles_data(subject_id).T

    # Remove the two standin regions.
    sc = np.delete(sc, 51, 0)
    sc = np.delete(sc, 51, 1)
    sc = np.delete(sc, 0, 0)
    sc = np.delete(sc, 0, 1)
    micro_profiles = np.delete(micro_profiles, 51, 0)
    micro_profiles = np.delete(micro_profiles, 0, 0)

    return sc, micro_profiles

In [ ]:
def micro_profiles_transform(micro_profiles, sc):
    return np.kron(micro_profiles, micro_profiles), sc.flatten()

In [ ]:
import lin_gen_model

def train(subject_id, sc, micro_profiles):
    subject = lin_gen_model.Subject(subject_id, micro_profiles,sc, micro_profiles_transform)

    #Train the subject model.
    alpha_grid = lin_gen_model.calc_alpha_grid(subject.transformed_sc, subject.transformed_fc)
    alpha, rules, intercept = lin_gen_model.bic_selection(subject.transformed_sc, subject.transformed_fc, \
                                        lin_gen_model.lasso_regression, alpha_grid, intercept=True)

    # Make sure the output directory exists. If not created it.
    outdir = "lasso_micro_profile_to_sc_w_intercepts/"
    inout.check_paths(outdir)

    # Save the model and training info to seperate files.
    np.savetxt(outdir + f"rules{subject_id}", rules)
    #np.savetxt(outdir + f"intercepts{subject_id}", np.array((intercept)))
    with open(f"micro_profile_to_sc_subject_{subject_id}_training_info.txt", mode="w+", encoding="utf-8") as info_file:
        info_file.write(f"Optimal alpha for subject {subject_id} is {alpha}.")

In [ ]:
for subject_id in range(1, 5):
    sc, micro_profiles = get_data(subject_id)
    
    train(subject_id, sc, micro_profiles)

In [ ]:
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.metrics import r2_score

def create_r_squared_plots(sc, sc_pred):
    r_squared = np.column_stack((sc.flatten(), sc_pred.flatten()))
    r_squared = pd.DataFrame(r_squared, columns=('SC', 'Predicted SC'))

    r_squared_score = r2_score(r_squared['SC'], r_squared['Predicted SC'])
    print(f"R Squared: {r_squared_score}")

    spearman_correlation = stats.spearmanr(r_squared['SC'], r_squared['Predicted SC'])
    print(f"Spearman Correlation: {spearman_correlation[0]}")

    sns.jointplot(data=r_squared, x="SC", y="Predicted SC", kind="reg")
    plt.show()
    plt.close()

In [ ]:
def determine_error(subject_id):
    rules_path = f"lasso_micro_profile_to_sc/rules{subject_id}"
    rules = np.loadtxt(rules_path)
    rules = np.reshape(rules, (14, 14))

    sc_pred = micro_profiles @ rules @ micro_profiles.T
    rules_analytic = np.linalg.pinv(micro_profiles) @ sc_pred @ np.linalg.pinv(micro_profiles).T
    sc_pred_analytic = micro_profiles @ rules_analytic @ micro_profiles.T

    create_r_squared_plots(sc, sc_pred)
    create_r_squared_plots(sc, sc_pred_analytic)
    
    
    inout.heatmaps(rules, f"Rules {subject_id}", True)
    plt.show()
    plt.close()

    sc_pred = micro_profiles @ rules @ micro_profiles.T

    inout.heatmaps(sc_pred, f"Predicted SC {subject_id}", False)
    plt.show()
    plt.close()

    inout.heatmaps(sc, f"SC {subject_id}", False)
    plt.show()
    plt.close()


In [ ]:

print("Mean SC: ")
print(np.mean(sc))
print("Max SC: ")
print(np.max(sc))

In [ ]:
for subject_id in range(1, 5):
    determine_error(subject_id)
    print()